In [1]:
import os
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import tensorflow.contrib.keras as tf_keras

In [2]:
# configst
tfe.enable_eager_execution()
%matplotlib inline

In [3]:
# load data
dataset=dataset.load_dataset("./ShapeNet",num_of_examples=2)
X=tf.convert_to_tensor(dataset[0],dtype=tf.float32)
Y=None;

[load_dataset] loading from ./ShapeNet
[construct_paths] parsing dir ./ShapeNet for ['.obj', '.mtl'] ...
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/647576c71cff90b4ca8607f540cc62ba ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/647576c71cff90b4ca8607f540cc62ba ... 
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/dca7f88e151f196b8179909e06795c03 ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/dca7f88e151f196b8179909e06795c03 ... 
... done


In [4]:
# net parameters
conv_filter_count = [96, 128, 256, 256, 256, 256]
deconv_filter_count = [128, 128, 128, 64, 32, 2]

In [5]:
# encoder network
print(X.shape)
cur_tensor=X
k_s = [3,3]
for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    with tf.name_scope("encoder_layer"):
        cur_tensor=tf_keras.layers.Conv2D(filters=conv_filter_count[i],kernel_size= k_s)(cur_tensor)
        cur_tensor=tf_keras.layers.MaxPooling2D()(cur_tensor)
        cur_tensor=tf_keras.layers.Activation('relu')(cur_tensor)
    print(cur_tensor.shape)

(10, 512, 512, 3)
(10, 255, 255, 96)
(10, 126, 126, 128)
(10, 62, 62, 256)
(10, 30, 30, 256)
(10, 14, 14, 256)
(10, 6, 6, 256)


In [8]:
# flatten tensor before 3D-LSTM cells
render_count=cur_tensor.get_shape().as_list()[0]
cur_tensor=tf.reshape(cur_tensor,[render_count,-1])
print(cur_tensor.shape)
cur_tensor=tf_keras.layers.Dense(1024)(cur_tensor)
print(cur_tensor.shape)

(10, 9216)
(10, 1024)


In [9]:
# 3D-LSTM 
# construct 3D grid of cells
cell_grid= utils.grid3D(tf.keras.layers.GRU(1024),N=3)
out=[];
with tf.name_scope("3D-LSTM"):
    for cell in np.nditer(cell_grid,flags=['refs_ok']):
        out.append((cell.item())(cur_tensor));
        
out=np.array(out)

ValueError: Input 0 of layer gru_1 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [10, 1024]

In [ ]:
# decoder network
for i in range(6):
    with tf.name_scope("decoder_layer"):
        cur_tensor=Deconv3D(filters=deconv_filter_count[i],kernel_size=(3,3,3))(cur_tensor)
        cur_tensor=UpSampling3D()(cur_tensor)
        cur_tensor=Activation('relu')(cur_tensor)
        print(cur_tensor.shape)
    
print(cur_tensor.shape())

In [ ]:
net = Model(input=input_tensor,output=cur_tensor)
net.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
os.system("rm -rf ./logs/*")
tensor_board=TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
net.fit(X,np.random.rand(len(X),1,1024),epochs=1,callbacks=[tensor_board])